# LangChain从入门到实践：数据连接与流程编排

在上一篇文章中，我们介绍了LangChain的基础组件，包括模型封装、Prompt模板、结构化输出和Function Calling。本篇文章将继续深入，为大家介绍LangChain的数据连接能力和流程编排功能，帮助你构建更复杂的大模型应用。

注意事项：
运行本文代码前，请确保已安装必要的依赖包：pip install langchain-openai langchain-text-splitters langchain-community faiss-cpu pymupdf langchain-dashscope
请在.env文件中配置相应的API密钥
示例中的文件路径需要替换为你自己的文件路径

## 1. 数据连接封装

大模型应用通常需要连接外部数据源，LangChain提供了丰富的数据加载和处理组件，让我们可以轻松处理各种格式的文档。

### 1.1 文档加载器：Document Loaders

文档加载器负责从不同来源加载文档，以下是使用PyMuPDF加载PDF文档的示例：

In [1]:
# 安装必要的依赖
# %pip install pymupdf langchain-openai langchain-text-splitters faiss-cpu

from langchain_community.document_loaders import PyMuPDFLoader

# 加载PDF文档
loader = PyMuPDFLoader("./data/rag_data/llama2.pdf")
pages = loader.load_and_split()

# 打印第一页内容
print(pages[0].page_content)

Llama 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗
Louis Martin†
Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov

LangChain支持多种文档格式，包括PDF、Word、CSV、HTML等，你可以根据需要选择合适的加载器。

### 1.2 文本分割器

文本分割器用于将长文档切分成适合大模型处理的小块：

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 创建文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,         # 每个文本块的目标大小（字符数）
    chunk_overlap=100,      # 相邻文本块之间的重叠部分大小
    length_function=len,    # 用于计算文本长度的函数
    add_start_index=True,   # 是否添加起始索引信息
)

# 分割文本
paragraphs = text_splitter.create_documents([pages[0].page_content])
for para in paragraphs:
    print(para.page_content)
    print('-------')

Llama 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗
Louis Martin†
Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
-------
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
-------
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
-------
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Sa

### 1.3 向量数据库与向量检索

LangChain提供了与多种向量数据库的集成，方便实现语义搜索：

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader

# 加载文档
loader = PyMuPDFLoader("./data/rag_data/llama2.pdf")
pages = loader.load_and_split()

# 文档切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

texts = text_splitter.create_documents(
    [page.page_content for page in pages[:4]]
)

# 创建向量数据库
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
db = FAISS.from_documents(texts, embeddings)

# 检索相关文档
retriever = db.as_retriever(search_kwargs={"k": 3})  # 检索top-3相关文档
query = "llama2有多少参数?"
docs = retriever.invoke(query)

# 打印检索结果
for doc in docs:
    print(doc.page_content)
    print("----")

but are not releasing.§
2. Llama 2-Chat, a ﬁne-tuned version of Llama 2 that is optimized for dialogue use cases. We release
variants of this model with 7B, 13B, and 70B parameters as well.
We believe that the open release of LLMs, when done safely, will be a net beneﬁt to society. Like all LLMs,
----
Llama 2-Chat, at scales up to 70B parameters. On the series of helpfulness and safety benchmarks we tested,
Llama 2-Chat models generally perform better than existing open-source models. They also appear to
----
Sergey Edunov
Thomas Scialom∗
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
----


## 2. Chain和LangChain Expression Language (LCEL)

Chain是LangChain的核心概念之一，它允许我们将多个组件连接起来，形成一个完整的处理流程。LangChain Expression Language (LCEL)是LangChain提供的一种声明式语言，用于简化Chain的构建过程。

### 2.1 使用LCEL构建简单Pipeline

LCEL允许我们使用管道符|将不同组件连接起来，形成处理流程：

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field, validator
from typing import List, Dict, Optional
from enum import Enum
import json
from langchain.chat_models import init_chat_model

# 定义输出结构
class SortEnum(str, Enum):
    data = 'data'   # 按流量排序
    price = 'price' # 按价格排序

class OrderingEnum(str, Enum):
    ascend = 'ascend'   # 升序
    descend = 'descend' # 降序

class Semantics(BaseModel):
    name: Optional[str] = Field(description="流量包名称", default=None)
    price_lower: Optional[int] = Field(description="价格下限", default=None)
    price_upper: Optional[int] = Field(description="价格上限", default=None)
    data_lower: Optional[int] = Field(description="流量下限", default=None)
    data_upper: Optional[int] = Field(description="流量上限", default=None)
    sort_by: Optional[SortEnum] = Field(description="按价格或流量排序", default=None)
    ordering: Optional[OrderingEnum] = Field(description="升序或降序排列", default=None)

# Prompt模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个语义解析器。你的任务是将用户的输入解析成JSON表示。不要回答用户的问题。"),
    ("human", "{text}"),
])

# 模型
llm = init_chat_model("gpt-4o", model_provider="openai")
structured_llm = llm.with_structured_output(Semantics)

# LCEL表达式
runnable = (
    {"text": RunnablePassthrough()} | prompt | structured_llm
)

# 运行
ret = runnable.invoke("不超过100元的流量大的套餐有哪些")
print(json.dumps(ret.model_dump(), indent=4, ensure_ascii=False))

{
    "name": null,
    "price_lower": null,
    "price_upper": 100,
    "data_lower": null,
    "data_upper": null,
    "sort_by": "data",
    "ordering": "descend"
}


### 2.2 使用LCEL实现RAG

RAG (Retrieval-Augmented Generation) 是大模型应用的一种重要模式，LCEL可以帮助我们轻松实现：

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 加载文档
loader = PyMuPDFLoader("./data/rag_data/llama2.pdf")
pages = loader.load_and_split()

# 文档切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

texts = text_splitter.create_documents(
    [page.page_content for page in pages[:4]]
)

# 创建向量数据库
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 2})

# Prompt模板
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 构建RAG Chain
rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt
    | llm
    | StrOutputParser()
)

# 运行RAG
answer = rag_chain.invoke("文档中有什么重要信息")
print(answer)

根据提供的内容，文档中列出了以下部分的重要信息：

1. **数据预训练（Pretraining）**：
   - 前馈数据（Pretraining Data）
   - 训练细节（Training Details）

2. **附录部分**：
   - 数据标注（Data Annotation）
   - 数据集污染（Dataset Contamination）
   - 模型卡（Model Card）

这些部分涉及到模型的训练过程及相关数据的信息。


### 2.3 使用LCEL实现工厂模式

LCEL还支持工厂模式，可以根据需要动态选择使用的模型：

In [10]:
%pip install langchain-dashscope

   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   - -------------------------------------- 20.5/798.9 kB ? eta -:--:--
   - -------------------------------------- 20.5/798.9 kB ? eta -:--:--
   - ------------------------------------- 30.7/798.9 kB 262.6 kB/s eta 0:00:03
   - ------------------------------------- 30.7/798.9 kB 262.6 kB/s eta 0:00:03
   - ------------------------------------- 41.0/798.9 kB 178.6 kB/s eta 0:00:05
   - ------------------------------------- 41.0/798.9 kB 178.6 kB/s eta 0:00:05
   -- ------------------------------------ 61.4/798.9 kB 218.8 kB/s eta 0:00:04
   -- ------------------------------------ 61.4/798.9 kB 218.8 kB/s eta 0:00:04
   --- ----------------------------------- 81.9/798.9 kB 229.0 kB/s eta 0:00:04
   ----- -------------------------------- 112.6/798.9 kB 297.7 kB/s eta 0:00:03
   

  You can safely remove it manually.


In [15]:
%pip install qianfan

  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
   ---------------------------------------- 0.0/470.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/470.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/470.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/470.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/470.3 kB ? eta -:--:--
   --- ------------------------------------ 41.0/470.3 kB 1.9 MB/s eta 0:00:01
   --- ------------------------------------ 41.0/470.3 kB 1.9 MB/s eta 0:00:01
   ---- ---------------------------------- 51.2/470.3 kB 435.7 kB/s eta 0:00:01
   ----- --------------------------------- 71.7/470.3 kB 491.5 kB/s eta 0:00:01
   ------ -------------------------------- 81.9/470.3 kB 381.3 kB/s eta 0:00:02
   --------- ---------------------------- 122.9/470.3 kB 481.4 kB/s eta 0:00:01
   ---------- --------------------------- 133.1/470.3 kB 462.0 kB/s eta 0:00:01
   ------------ ------

In [18]:
from langchain_core.runnables.utils import ConfigurableField
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_community.chat_models import QianfanChatEndpoint  # 使用QianfanChatEndpoint
import os
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

# 初始化不同的模型
ds_model = init_chat_model("deepseek-chat", model_provider="deepseek")
gpt_model = init_chat_model("gpt-4o-mini", model_provider="openai")

# 配置可选模型
model = gpt_model.configurable_alternatives(
    ConfigurableField(id="llm"), 
    default_key="gpt",  # 设置默认模型为GPT
    deepseek=ds_model,
    # 可以添加更多模型
)

# Prompt模板
prompt = ChatPromptTemplate.from_messages([
    ("human", "{query}"),
])

# 构建Chain
chain = (
    {"query": RunnablePassthrough()} 
    | prompt
    | model 
    | StrOutputParser()
)

# 运行时指定模型
ret = chain.with_config(configurable={"llm": "deepseek"}).invoke("请自我介绍")
print(ret)

你好！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的智能AI助手。我可以帮助你解答各种问题，包括学习、工作、编程、写作、翻译、生活建议等。  

### **我的特点**：  
🔹 **免费使用**：目前无需付费，随时为你提供帮助！  
🔹 **超长上下文**：支持 **128K** 上下文记忆，能处理超长文档和复杂对话。  
🔹 **多文档处理**：可以上传 **PDF、Word、Excel、PPT、TXT** 等文件，并帮你提取和分析内容。  
🔹 **知识丰富**：我的知识截止到 **2024年7月**，能提供较新的信息。  
🔹 **多语言支持**：可以用中文、英文等多种语言交流。  

### **我能帮你做什么？**  
📚 **学习**：解题思路、论文润色、知识讲解  
💼 **工作**：简历优化、邮件撰写、数据分析  
💻 **编程**：代码调试、算法讲解、技术文档解读  
✍️ **写作**：创意写作、文案优化、故事构思  
🌍 **生活**：旅行建议、健康小贴士、娱乐推荐  

你可以随时向我提问，我会尽力提供准确、有用的答案！😊 有什么我可以帮你的吗？


通过LCEL，我们还可以实现更多高级功能：

配置运行时变量
故障回退
并行调用
逻辑分支
动态创建Chain

总结
在本文中，我们介绍了LangChain的数据连接能力和流程编排功能。这些功能为构建复杂的大模型应用提供了强大支持：
数据连接封装：提供了各种文档加载器、文本分割器和向量存储接口，方便处理各种格式的文档和实现语义搜索。
LCEL：提供了一种声明式语言，简化了Chain的构建过程，支持各种复杂的应用模式，如RAG、工厂模式等。

在下一篇文章中，我们将介绍LangChain的工作流框架LangGraph，它为构建更复杂的大模型应用提供了更高级的支持。